<h1>Import libraries</h1>

In [4]:
pip install fosforml

     |████████████████████████████████| 51kB 2.9MB/s eta 0:00:01
     |████████████████████████████████| 10.9MB 35.4MB/s eta 0:00:01
     |████████████████████████████████| 1.9MB 70.2MB/s eta 0:00:01
     |████████████████████████████████| 39.9MB 22.7MB/s eta 0:00:01███████████▊                    | 14.7MB 22.7MB/s eta 0:00:02
     |████████████████████████████████| 98.7MB 167kB/s  eta 0:00:01  |████▏                           | 12.8MB 76.0MB/s eta 0:00:02    |████████████████▍               | 50.5MB 76.0MB/s eta 0:00:01
     |████████████████████████████████| 200.3MB 80kB/s /s eta 0:00:01 44.0MB 41.6MB/s eta 0:00:04          | 85.7MB 41.6MB/s eta 0:00:03�██▉                | 99.1MB 113.8MB/s eta 0:00:01[K     |███████████████████             | 119.3MB 113.8MB/s eta 0:00:01
     |████████████████████████████████| 81kB 13.4MB/s eta 0:00:01
     |████████████████████████████████| 1.3MB 70.2MB/s eta 0:00:01
     |████████████████████████████████| 174kB 87.5MB/s eta 0:00:01
     |█████████

<h1>Establish connection to snowflake and Importing Tables</h1>

In [ ]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()
 
table_name = 'SOL_ASSORTMENT_PLANNING_COMB'
 
sf_df = my_session.sql("select * from {}".format(table_name))
sales_bangalore_2022 = sf_df.to_pandas()
 
sales_bangalore_2022.head()

In [3]:
table_name = 'SOL_ASSORTMENT_PLANNING_TRAIN_DATA_MODEL'
 
sf_df = my_session.sql("select * from {}".format(table_name))
train_final = sf_df.to_pandas()
 
train_final.head()

NameError: name 'my_session' is not defined

In [3]:
table_name = 'SOL_ASSORTMENT_PLANNING_TEST_SET_MODEL'
 
sf_df = my_session.sql("select * from {}".format(table_name))
test_final = sf_df.to_pandas()
 
test_final.head()

,S_NO,UNIQUE_ID,FREQUENCYM,SALES_VALUE_AVG,SALES_UNITS_AVG,SALES_INDICATOR,SCHEME_AMOUNT_PERPRODUCT
0,0,PRD0002-OL10330,0.666667,0.004323138,0.00040404,1,0.001560
1,1,PRD0002-OL10346,0.166667,0.001429593,0.00013468,0,0.001560
2,2,PRD0002-OL10347,0.166667,0.001766519,0.00016835,0,0.001559
3,3,PRD0002-OL10358,1.000000,0.01577123,0.001481481,1,0.001560
4,4,PRD0002-OL10386,0.833333,0.00841699,0.000791246,1,0.001560


<h1>Analyze data</h1>

In [4]:
train_final.columns = train_final.columns.str.lower()

In [5]:
train_final.head(1)

,s_no,unique_id,frequencym,sales_value_avg,sales_units_avg,sales_indicator,scheme_amount_perproduct
0,0,PRD0002-OL10330,0.6,0.004323138,0.00040404,1,0.000465


In [6]:
master_data = sales_bangalore_2022.copy()
master_data.head(1)

,TRANSACTION_DATE,MNTH_CODE,YEAR_CODE,SALES_VALUE,SALES_UNITS,SALES_VOLUME,SALES_PTR_VALUE,DISTRIBUTOR_CODE,PRODUCT_CODE,OUTLET_CODE,CITY,STATE,COUNTY,STREET,CATEGORY,VARIANT,BRAND
0,2023-07-01,202203,2022,460.91,3,0.0027,460.909091,DB0706,PRD0041,OL81278,Montclair,New Jersey,Dolphin,Str1,Soap,Beauty Soap,Charcoal


In [7]:
master_data.columns = master_data.columns.str.lower()
master_data.head(1)

,transaction_date,mnth_code,year_code,sales_value,sales_units,sales_volume,sales_ptr_value,distributor_code,product_code,outlet_code,city,state,county,street,category,variant,brand
0,2023-07-01,202203,2022,460.91,3,0.0027,460.909091,DB0706,PRD0041,OL81278,Montclair,New Jersey,Dolphin,Str1,Soap,Beauty Soap,Charcoal


In [8]:
df1=master_data.copy
master_data['unique_id']=master_data['product_code']+'-'+ master_data['outlet_code']

<h1>Train Model</h1>

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(train_final[['frequencym','sales_value_avg','scheme_amount_perproduct']],train_final['sales_indicator'],test_size=0.3,random_state=42)

In [10]:
X_test.columns = X_test.columns.str.lower()

In [11]:
import pickle

with open('logreg.pkl', 'rb') as model_file:
   loaded_model = pickle.load(model_file)
probs = loaded_model.predict_proba(X_test)
probs

array([[0.66778068, 0.33221932],
       [0.8336751 , 0.1663249 ],
       [0.44592937, 0.55407063],
       ...,
       [0.66800724, 0.33199276],
       [0.83363294, 0.16636706],
       [0.6676696 , 0.3323304 ]])

In [12]:
cutoff = 0.3
y_pred = (probs[:,1]>=cutoff).astype(int)
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.83      0.60      0.70     30799
           1       0.57      0.82      0.68     20399

    accuracy                           0.69     51198
   macro avg       0.70      0.71      0.69     51198
weighted avg       0.73      0.69      0.69     51198



<h1> Test model </h1>

In [13]:
test_final.columns = test_final.columns.str.lower()
test_final.head(1)

,s_no,unique_id,frequencym,sales_value_avg,sales_units_avg,sales_indicator,scheme_amount_perproduct
0,0,PRD0002-OL10330,0.666667,0.004323138,0.00040404,1,0.00156


In [14]:
X_test_unseen = test_final[['frequencym','sales_value_avg','scheme_amount_perproduct']]
y_test_unseen = test_final['sales_indicator']
print(X_test_unseen.shape,y_test_unseen.shape)
probs = loaded_model.predict_proba(X_test_unseen)
test_final['probablity']=probs[:,1]
y_pred = (probs[:,1]>=0.4).astype(int)
test_final

(189082, 3) (189082,)


,s_no,unique_id,frequencym,sales_value_avg,sales_units_avg,sales_indicator,scheme_amount_perproduct,probablity
0,0,PRD0002-OL10330,0.666667,0.004323138,0.00040404,1,0.001560,0.807117
1,1,PRD0002-OL10346,0.166667,0.001429593,0.00013468,0,0.001560,0.298748
2,2,PRD0002-OL10347,0.166667,0.001766519,0.00016835,0,0.001559,0.298690
3,3,PRD0002-OL10358,1.000000,0.01577123,0.001481481,1,0.001560,0.950099
4,4,PRD0002-OL10386,0.833333,0.00841699,0.000791246,1,0.001560,0.899377
...,...,...,...,...,...,...,...,...
189077,189077,PRD0168-OL97586,0.166667,0.001948381,0.00030303,1,0.001560,0.298657
189078,189078,PRD0168-OL97590,0.166667,0,0,0,0.001560,0.298999
189079,189079,PRD0168-OL97602,0.000000,0.001581375,0.000242424,0,0.001560,0.165792
189080,189080,PRD0168-OL97615,0.333333,0.001319099,0.00020202,1,0.001560,0.477312


In [15]:
test_final['prediction']= test_final['probablity'].apply(lambda x: 1  if x>=0.3 else 0)
test_final

,s_no,unique_id,frequencym,sales_value_avg,sales_units_avg,sales_indicator,scheme_amount_perproduct,probablity,prediction
0,0,PRD0002-OL10330,0.666667,0.004323138,0.00040404,1,0.001560,0.807117,1
1,1,PRD0002-OL10346,0.166667,0.001429593,0.00013468,0,0.001560,0.298748,0
2,2,PRD0002-OL10347,0.166667,0.001766519,0.00016835,0,0.001559,0.298690,0
3,3,PRD0002-OL10358,1.000000,0.01577123,0.001481481,1,0.001560,0.950099,1
4,4,PRD0002-OL10386,0.833333,0.00841699,0.000791246,1,0.001560,0.899377,1
...,...,...,...,...,...,...,...,...,...
189077,189077,PRD0168-OL97586,0.166667,0.001948381,0.00030303,1,0.001560,0.298657,0
189078,189078,PRD0168-OL97590,0.166667,0,0,0,0.001560,0.298999,0
189079,189079,PRD0168-OL97602,0.000000,0.001581375,0.000242424,0,0.001560,0.165792,0
189080,189080,PRD0168-OL97615,0.333333,0.001319099,0.00020202,1,0.001560,0.477312,1


In [16]:
test_final_static = test_final.copy()
print(classification_report(test_final['sales_indicator'],test_final['prediction']))

              precision    recall  f1-score   support

           0       0.79      0.75      0.77    124905
           1       0.56      0.61      0.58     64177

    accuracy                           0.70    189082
   macro avg       0.67      0.68      0.68    189082
weighted avg       0.71      0.70      0.71    189082



In [17]:
test_final.reset_index(drop=True)
test_final['OOS'] = test_final['probablity'].apply(lambda x:1 if x>0.3 else 0)

In [18]:
import pandas as pd

In [19]:
soq_master = master_data.groupby('unique_id').agg({'sales_units':'sum','mnth_code':'nunique'})
soq_master['SOQ'] = soq_master['sales_units']/soq_master['mnth_code']
soq_master.reset_index()
final_reco = pd.merge(soq_master,test_final,on='unique_id',how='inner')

In [20]:
final_reco = final_reco[['unique_id','prediction','SOQ','OOS']]
final_reco.head()

,unique_id,prediction,SOQ,OOS
0,PRD0002-OL10330,1,2.714286,1
1,PRD0002-OL10346,0,1.500000,0
2,PRD0002-OL10347,0,1.250000,0
3,PRD0002-OL10358,1,7.166667,1
4,PRD0002-OL10386,1,4.090909,1


In [21]:
ms_reco = final_reco.copy()
ms_reco.columns = ['unique_id', 'ms_flag', 'soq', 'oos_flag']

<h1> Write output to a snowflake table </h1>

In [22]:
ms_reco[['product', 'outlet']] = ms_reco['unique_id'].str.split('-', expand=True)
ms_reco.head()

,unique_id,ms_flag,soq,oos_flag,product,outlet
0,PRD0002-OL10330,1,2.714286,1,PRD0002,OL10330
1,PRD0002-OL10346,0,1.500000,0,PRD0002,OL10346
2,PRD0002-OL10347,0,1.250000,0,PRD0002,OL10347
3,PRD0002-OL10358,1,7.166667,1,PRD0002,OL10358
4,PRD0002-OL10386,1,4.090909,1,PRD0002,OL10386


In [23]:
ms_reco.columns = ms_reco.columns.str.upper()

In [24]:
sf_df = my_session.createDataFrame(ms_reco)
sf_df.write.mode("overwrite").save_as_table("SOL_ASSORTMENT_PREDICTION_TABLE")
my_session.table("SOL_ASSORTMENT_PREDICTION_TABLE").show()

----------------------------------------------------------------------------------------
|"UNIQUE_ID"      |"MS_FLAG"  |"SOQ"               |"OOS_FLAG"  |"PRODUCT"  |"OUTLET"  |
----------------------------------------------------------------------------------------
|PRD0002-OL10330  |1          |2.7142857142857144  |1           |PRD0002    |OL10330   |
|PRD0002-OL10346  |0          |1.5                 |0           |PRD0002    |OL10346   |
|PRD0002-OL10347  |0          |1.25                |0           |PRD0002    |OL10347   |
|PRD0002-OL10358  |1          |7.166666666666667   |1           |PRD0002    |OL10358   |
|PRD0002-OL10386  |1          |4.090909090909091   |1           |PRD0002    |OL10386   |
|PRD0002-OL10408  |0          |2.5                 |0           |PRD0002    |OL10408   |
|PRD0002-OL10472  |1          |3.1666666666666665  |1           |PRD0002    |OL10472   |
|PRD0002-OL10525  |1          |3.3333333333333335  |1           |PRD0002    |OL10525   |
|PRD0002-OL10545  |1 

In [25]:
X_train.head()

,frequencym,sales_value_avg,scheme_amount_perproduct
10793,0.6,0.003147963,0.000465
74989,1.0,0.00123731,0.000516
132485,0.6,0.000679882,0.000603
59852,0.0,0,0.000465
148862,0.2,0.000729096,0.000465


In [26]:
y_test.head()

148184    1
116643    0
120336    0
57315     0
54434     0
Name: sales_indicator, dtype: int8

In [30]:
y_pred = pd.Series(y_pred)
y_pred.name = "SALES_PREDICTED"

In [ ]:
type(y_pred)

In [31]:
dataset_source="InMemory"
 

<h1>Register model in FDC</h1>

In [32]:
from fosforml import register_model

In [33]:
register_model(
    model_obj=loaded_model,
    session=my_session,
    x_train=X_train,
    y_train=y_train,
    x_test=X_test,
    y_test=y_test,
    y_pred=y_pred,
    source="Notebook",
    dataset_name="SOL_ASSORTMENT_PLANNING_TEST_SET_MODEL",
    dataset_source="InMemory",
    name="Assortment_Planning_Prediction",
    description="prediction of assortment across retailers by product",
    flavour="sklearn",
    model_type="regression",
    conda_dependencies=[]
)
 

Got error object of type 'NoneType' has no len() when trying to read default values from function: <function explained_variance_score.<locals>.explained_variance_score_anon_sproc at 0x7fce372e74c0>. Proceeding without creating optional arguments


Calculating build time metrics

Progress: ██████████████████                                                     25.0%


The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn==1.3.*'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'snowflake-snowpark-python' in the local environment is 1.21.0, which does not fit the criteria for the requirement 'snowflake-snowpark-python'. Your UDF might not work when the package version is different between the server and your local environment.


(1300) (1304): 01b6b1f2-070e-1713-0072-f30310493076: 100357 (P0000): Python Interpreter Error:
Traceback (most recent call last):
  File "/home/udf/32355341556222970/udf_py_567825513.zip/udf_py_567825513.py", line 53, in compute
    return func(session)
  File "/opt/conda/lib/python3.9/site-packages/snowflake/ml/modeling/metrics/regression.py", line 314, in explained_variance_score_anon_sproc
  File "/usr/lib/python_udf/28d49ae176353f9c4fce948a3eccb91c4a6655effb2197b174c162742c2e895f/lib/python3.9/site-packages/sklearn/utils/_param_validation.py", line 211, in wrapper
    return func(*args, **kwargs)
  File "/usr/lib/python_udf/28d49ae176353f9c4fce948a3eccb91c4a6655effb2197b174c162742c2e895f/lib/python3.9/site-packages/sklearn/metrics/_regression.py", line 826, in explained_variance_score
    y_type, y_true, y_pred, multioutput = _check_reg_targets(
  File "/usr/lib/python_udf/28d49ae176353f9c4fce948a3eccb91c4a6655effb2197b174c162742c2e895f/lib/python3.9/site-packages/sklearn/metrics/_

"Model 'MODEL_F15A51B3_9BD9_4881_B1ED_A675DCDCE2B2_FDC_ASSORTMENT_PLANNING_PREDICTION' registered successfully."